# Data Collector

This notebook organizes and aggregates Excel workbooks of faculty diversity data.

In [38]:
import os
import pandas as pd

The following dictionary maps my custom filename convention to metadata.

In [62]:
META = {
    'hum': 'Humanities',
    'ns': 'Natural Sciences',
    'ss': 'Social Sciences',
    '1': 'Not eligible for tenure',
    '2': 'Not tenured, but eligible for tenure',
    '3': 'Tenured',    
}

In [102]:
DATA_PATH = '../data/excel/'

def getMetadata(fname):
    [div, status] = fname.rstrip('.xlsx').split('-')
    div = META[div]
    status = META[status]
    return div, status

def getDF(fname):
    df = pd.read_excel(DATA_PATH + fname)
    div, status = getMetadata(fname)
    df.columns = [
        'Year', 
        '% Women', 
        'Women', 
        'Total', 
        '% Men',
        'Men', 
        'Total Duplicate'
    ]
    df.drop(columns=['Total Duplicate'], inplace=True)
    df.set_index('Year', inplace=True)
    df.division, df.status = div, status
    df.name = '{}: {}'.format(div, status)
    return df

In [104]:
for fname in os.listdir(DATA_PATH):
    df = getDF(fname)
    print(df.name)
    print(df)

Humanities: Tenured
      % Women  Women  Total   % Men  Men
Year                                    
2007   0.3607     44    122  0.6393   78
2008   0.3835     51    133  0.6165   82
2009   0.3759     53    141  0.6241   88
2010   0.4000     54    135  0.6000   81
2011   0.3929     55    140  0.6071   85
2012   0.3944     56    142  0.6056   86
2013   0.3946     58    147  0.6054   89
2014   0.3907     59    151  0.6093   92
2015   0.3947     60    152  0.6053   92
2016   0.4013     61    152  0.5987   91
2017   0.4183     64    153  0.5817   89
Social Sciences: Not eligible for tenure
      % Women  Women  Total   % Men  Men
Year                                    
2007   0.4500      9     20  0.5500   11
2008   0.5263     10     19  0.4737    9
2009   0.4706      8     17  0.5294    9
2010   0.4762     10     21  0.5238   11
2011   0.5000     11     22  0.5000   11
2012   0.6000     12     20  0.4000    8
2013   0.5588     19     34  0.4412   15
2014   0.4848     16     33  0.5152  

In [ ]:
a = getDF('hum-1.xlsx')
b = getDF('hum-2.xlsx')